In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("My Spark Application") \
    .getOrCreate()

131072x1 화면 크기가 잘못됐습니다. 문제가 예상됩니다
2023-12-07 16:46:27,350 WARN util.Utils: Your hostname, KJH resolves to a loopback address: 127.0.1.1; using 172.24.249.89 instead (on interface eth0)
2023-12-07 16:46:27,351 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-12-07 16:46:27,950 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [48]:
df_genome_scores = spark.read.csv("file:///home/kjh/data/ml-latest/genome-scores.csv", header=True, inferSchema=True)
df_genome_tags = spark.read.csv("file:///home/kjh/data/ml-latest/genome-tags.csv", header=True, inferSchema=True)
df_movies = spark.read.csv("file:///home/kjh/data/ml-latest/movies.csv", header=True, inferSchema=True)
df_links = spark.read.csv("file:///home/kjh/data/ml-latest/links.csv", header=True, inferSchema=True)
df_ratings = spark.read.csv("file:///home/kjh/data/ml-latest/ratings.csv", header=True, inferSchema=True)
df_tags = spark.read.csv("file:///home/kjh/data/ml-latest/tags.csv", header=True, inferSchema=True)

In [49]:
df_links.show(4)
df_ratings.show(4)
df_tags.show(4)

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
|      4|114885| 31357|
+-------+------+------+
only showing top 4 rows

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      1|   4.0|1225734739|
|     1|    110|   4.0|1225865086|
|     1|    158|   4.0|1225733503|
|     1|    260|   4.5|1225735204|
+------+-------+------+----------+
only showing top 4 rows

+------+-------+-------------+----------+
|userId|movieId|          tag| timestamp|
+------+-------+-------------+----------+
|    10|    260| good vs evil|1430666558|
|    10|    260|Harrison Ford|1430666505|
|    10|    260|       sci-fi|1430666538|
|    14|   1221|    Al Pacino|1311600756|
+------+-------+-------------+----------+
only showing top 4 rows



In [4]:
df_genome_scores.show(4)
df_genome_tags.show(4)
df_movies.show(4)

+-------+-----+--------------------+
|movieId|tagId|           relevance|
+-------+-----+--------------------+
|      1|    1| 0.03199999999999997|
|      1|    2|0.022249999999999992|
|      1|    3|                0.07|
|      1|    4|               0.059|
+-------+-----+--------------------+
only showing top 4 rows

+-----+------------+
|tagId|         tag|
+-----+------------+
|    1|         007|
|    2|007 (series)|
|    3|18th century|
|    4|       1920s|
+-----+------------+
only showing top 4 rows

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
+-------+--------------------+--------------------+
only showing top 4 rows



In [5]:
df_genome_scores.createOrReplaceTempView("scores")
df_genome_tags.createOrReplaceTempView("tags")
df_movies.createOrReplaceTempView("movies")

In [6]:
spark.sql("SELECT COUNT(*) FROM scores").show()
spark.sql("SELECT COUNT(*) FROM tags").show()
spark.sql("SELECT COUNT(*) FROM movies").show()

+--------+
|count(1)|
+--------+
|18472128|
+--------+

+--------+
|count(1)|
+--------+
|    1128|
+--------+

+--------+
|count(1)|
+--------+
|   86537|
+--------+



In [12]:
spark.sql("""SELECT m.movieId, m.title, m.genres, s.tagId, t.tag, s.relevance
          FROM scores s
          JOIN tags t ON s.tagId = t.tagId
          JOIN movies m ON s.movieId = m.movieId 
          """).show()

+-------+----------------+--------------------+-----+---------------+--------------------+
|movieId|           title|              genres|tagId|            tag|           relevance|
+-------+----------------+--------------------+-----+---------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|    1|            007| 0.03199999999999997|
|      1|Toy Story (1995)|Adventure|Animati...|    2|   007 (series)|0.022249999999999992|
|      1|Toy Story (1995)|Adventure|Animati...|    3|   18th century|                0.07|
|      1|Toy Story (1995)|Adventure|Animati...|    4|          1920s|               0.059|
|      1|Toy Story (1995)|Adventure|Animati...|    5|          1930s|               0.123|
|      1|Toy Story (1995)|Adventure|Animati...|    6|          1950s|               0.131|
|      1|Toy Story (1995)|Adventure|Animati...|    7|          1960s| 0.06175000000000003|
|      1|Toy Story (1995)|Adventure|Animati...|    8|          1970s|              0.1955|

In [22]:
spark.sql("""SELECT m.movieId, m.title, m.genres, s.tagId, t.tag, s.relevance
          FROM scores s
          JOIN tags t ON s.tagId = t.tagId
          JOIN movies m ON s.movieId = m.movieId 
          WHERE t.tag IS NOT NULL
          """).count()

18472128

In [47]:
spark.sql("""
          SELECT DISTINCT movieId
          FROM (
            SELECT movieId, COUNT(*) OVER (PARTITION BY movieId) as cnt, tagId, relevance, max_relevance
            FROM (
                SELECT movieId, tagId, relevance,
                    MAX(relevance) OVER (PARTITION BY movieId) as max_relevance
                FROM scores
                ) 
                WHERE relevance = max_relevance
            )
            WHERE cnt > 1    
            """).count()

115

In [38]:
spark.sql("""
    SELECT COUNT(DISTINCT movieId)
    FROM scores
""").show()


+-----------------------+
|count(DISTINCT movieId)|
+-----------------------+
|                  16376|
+-----------------------+



In [17]:
spark.sql("""
        SELECT m.movieId, m.title, m.genres, s.tagId, t.tag, s.relevance
        FROM (
                SELECT movieId, tagId, relevance,
                    MAX(relevance) OVER (PARTITION BY movieId) as max_relevance
                FROM scores
        ) s
        JOIN tags t ON s.tagId = t.tagId
        JOIN movies m ON s.movieId = m.movieId 
        WHERE s.relevance = s.max_relevance
        """).show()

+-------+--------------------+--------------------+-----+--------------------+------------------+
|movieId|               title|              genres|tagId|                 tag|         relevance|
+-------+--------------------+--------------------+-----+--------------------+------------------+
|     28|   Persuasion (1995)|       Drama|Romance|   22|   adapted from:book|            0.9895|
|     31|Dangerous Minds (...|               Drama| 1010|             teacher|             0.942|
|     34|         Babe (1995)|      Children|Drama| 1007|     talking animals|0.9850000000000001|
|     53|     Lamerica (1994)|     Adventure|Drama|  270|           criterion|            0.6835|
|     65|     Bio-Dome (1996)|              Comedy|  980|           stupidity|             0.974|
|     78|Crossing Guard, T...|Action|Crime|Dram...|  478|               guilt|             0.891|
|     81|Things to Do in D...| Crime|Drama|Romance|  269|     crime gone awry|0.9990000000000001|
|     85|Angels and 